Experimental condition 4: Appraiser's reconstruction is used to train appraiser

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import Tensor
import tensorflow_probability as tfp
import cv2
import glob
import gym
import random
from gym import Env
from gym.spaces import Discrete, Box
import math
import PIL
import PIL.ImageDraw
import PIL.Image
from PIL import ImageFont
import os
"""making tf allocate memory dynamically, to make the running of the program more efficiently"""
tf.compat.v1.enable_eager_execution()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

In [ ]:
"""creates relevant directory where model weights will be saved"""
os.mkdir('experimental_condition_four')

In [ ]:
"""selects 1000 images randomly from a directory of saved images, here images/images/"""
"""these images are used to train the autoencoder"""

import os
filenames = [img for img in glob.glob("images/images/*/*.jpg")]
np.random.shuffle(filenames)
filenames = filenames[:1000]
images = [cv2.imread(img) for img in filenames]
for i in range(0,len(images)):
    images[i] = cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB)
    images[i] = cv2.resize(images[i], (400,400))
images = np.array(images)
images = images.astype('float32')/255

In [2]:
"""custom layer which produces the distribution which is sampled in the VAE"""

class Latent_features(tf.keras.layers.Layer):
    def call(self, inputs) -> Tensor:
        dist_mean, dist_log_var = inputs
        batch = tf.shape(dist_mean)[0]
        dim = tf.shape(dist_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return dist_mean + tf.exp(0.5 * dist_log_var) * epsilon

In [3]:
class VAE(tf.keras.Model):
    
    """builds encoder and decoder sub-networks"""
    
    def __init__(self):
        super(VAE, self).__init__()
        self.conv_layer_steps = [2,2,2,2]
        self.encoder = self.build_encoder()
        self.decoder = self.build_decoder()
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        
        autoencoder_input = keras.Input(shape=(400,400,3))
        encoded_img = self.encoder(autoencoder_input)[2]
        decoded_img = self.decoder(encoded_img)
        self.autoencoder = keras.Model(autoencoder_input, decoded_img)
    
    """custom call function necessary for custom keras model"""
    def call(self, image):
        encoded = self.encoder(image)[2]
        decoded = self.decoder(encoded)
        return decoded
    
    """utility function for batch norm and leakyrelu"""
    def activation_block(self, inputs) -> Tensor:
        BN = tf.keras.layers.BatchNormalization(momentum=0.99) (inputs)
        activated = tf.keras.layers.LeakyReLU()(BN)
        return activated
    
    """implements skip connection for the resnet component of the architecture"""
    def residual_block(self, inputs: Tensor, scale_change: bool, polarity, kernel_size = 2, filters = 64, stride_size = 2) -> Tensor:
        #POLARITY IS 0 IF DOWNSCALING (ENCODER) 1 IF UPSCALING (DECODER)
        if polarity == 0:
            output_1 = tf.keras.layers.Conv2D(filters, (kernel_size, kernel_size), strides = (1 if not scale_change else stride_size), padding = 'same') (inputs)
            output_1 = self.activation_block(output_1)
            output_1 = tf.keras.layers.Conv2D(filters, (kernel_size, kernel_size), strides = (1), padding = 'same') (output_1)
            if scale_change == True:
                inputs = tf.keras.layers.Conv2D(filters, (kernel_size, kernel_size), strides = (stride_size), padding = 'same') (inputs)
        
        else:
            output_1 = tf.keras.layers.Conv2DTranspose(filters, (kernel_size, kernel_size), strides = (1 if not scale_change else stride_size), padding = 'same') (inputs)
            output_1 = self.activation_block(output_1)
            output_1 = tf.keras.layers.Conv2DTranspose(filters, (kernel_size, kernel_size), strides = (1), padding = 'same') (output_1)
            if scale_change == True:
                inputs = tf.keras.layers.Conv2DTranspose(filters, (kernel_size, kernel_size), strides = (stride_size), padding = 'same') (inputs)
        
        output_2 = tf.keras.layers.Add()([inputs, output_1])
        output_2 = self.activation_block(output_2)
        
        return(output_2)
        
    """returns the encoder model, a sub-architecture of the autoencoder"""
    def build_encoder(self):
        num_filters = 32
        conv_layer_steps = self.conv_layer_steps
        E_input = tf.keras.layers.Input(shape = (400,400,3), name = 'original_image')
        E = tf.keras.layers.Conv2D(num_filters, (1,1), strides=(1), padding = 'same') (E_input)
        E = self.activation_block(E)
        
        block_depths = [2,5,5,2]
        filters = [32,64,64,128]
        
        for i in range(0,len(block_depths)):
            for j in range(0,block_depths[i]):
                E = self.residual_block(E, (j==0 and i!=0), 0, 2, filters[i], 2)

        
            
        self.reshape_dims = E.shape
            
        E = tf.keras.layers.Flatten() (E)
        
        self.flatten_dims = E.shape
        
        E = (tf.keras.layers.LeakyReLU()) (E)
        
        distribution_mean = tf.keras.layers.Dense(16, name='mean')(E)
        distribution_variance = tf.keras.layers.Dense(16, name='log_variance')(E)
        latent_encoding = Latent_features()([distribution_mean, distribution_variance])
        
        
        encoder = keras.Model(E_input, [distribution_mean, distribution_variance, latent_encoding], name="encoder")
        
                
        return encoder
    
    """returns the decoder model, a sub-network for autoencoder architecture"""
    def build_decoder(self):
        num_filters = 256
        conv_layer_steps = self.conv_layer_steps
        decoder_input = keras.Input(shape=(16,))
        D = tf.keras.layers.LeakyReLU() (decoder_input)
        D = (tf.keras.layers.Dense(self.flatten_dims[1])) (D)
        D = (tf.keras.layers.LeakyReLU()) (D)
        D = (tf.keras.layers.Reshape((self.reshape_dims[1], self.reshape_dims[2], self.reshape_dims[3]))) (D)
        
        block_depths = [2,5,5,2]
        filters = [128,64,64,32]
        
        for i in range(0,len(block_depths)):
            for j in range(0,block_depths[i]):
                D = self.residual_block(D, (j==0 and i!=0), 1, 2, filters[i], 2)
            
        D = tf.keras.layers.Conv2DTranspose(3, (1,1), strides = (1), padding = 'same') (D)
        D = self.activation_block(D)
        
        
        decoded = (tf.keras.layers.Activation('sigmoid')) (D)
        
        decoder = tf.keras.Model(inputs = decoder_input, outputs = decoded)
        
        return decoder

    
    """necessary for loss trackers used for custom loss function"""
    @property
    def metrics(self):
        return [self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            mean, log_var, latent = self.encoder(data)
            reconstruction = self.decoder(latent)
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)))
            kl_loss = -0.5 * (1 + log_var - tf.square(mean) - tf.exp(log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {"loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),}


In [4]:
"""compiling the VAE with RMSprop, and a lower learning rate than during the VAE training loop"""
opt = tf.keras.optimizers.RMSprop(learning_rate = 0.0000005)
vae = VAE()

vae.compile(optimizer=opt)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [5]:
"""training VAE"""
vae.fit(images, epochs = 20, batch_size = 4)

In [6]:
class Painting_Env(gym.Env):
  def __init__(self, paintings_per_loop = 100, maximum_acceptable_loss = 0.005):

    super().__init__()
    self.action_space = Box(low = 0, high = 1, shape = (64,64,3))
    self.observation_space = Box(low = 0, high = 1, shape = (64,64,3))
    self.state = np.zeros((64,64,3))
    self.paintings_per_loop = paintings_per_loop
    self.paintings_per_loop_original = self.paintings_per_loop
    
    self.maximum_acceptable_loss = maximum_acceptable_loss



  def step(self, action):
    """resizing action to original_img to use on vae"""
    self.state = action.squeeze()
    original_img = cv2.resize(self.state, (400,400))

    current_weights = vae.get_weights()
    """creating reconstructed img"""
    reconstructed_orig = vae(original_img.reshape(1,400,400,3)).numpy().squeeze()
    """training vae on reconstructed img"""
    vae.train_step(reconstructed_orig.reshape(1,400,400,3))

    new_weights = vae.get_weights()

    weight_diff = []

    reconstructed = vae(reconstructed_orig.reshape(1,400,400,3)).numpy().squeeze()
    """finding weighted avg of weight differences"""
    for i in range(1,len(current_weights)-1):
      if current_weights[i].ndim != 0:
        weight_diff.append((tf.math.reduce_mean(np.abs(tf.keras.losses.MSE(current_weights[i],new_weights[i]).numpy())).numpy())*(len(current_weights)/2 - (abs(i-len(current_weights)/2))))
    """making this a single number"""
    weight_diff = tf.math.reduce_mean(weight_diff).numpy()

    loss_val = tf.math.reduce_mean(tf.keras.losses.MSE(original_img, reconstructed)).numpy()
    """if loss is above maximum acceptable loss, fix weight difference and revert changes to vae"""
    if loss_val > self.maximum_acceptable_loss:
      weight_diff = 0.0001
      vae.set_weights(current_weights)
    else:
      weight_diff *= 1e8

    self.reconstructed = reconstructed

    loss = (original_img - reconstructed)
    loss = np.abs(loss)
    """ensuring no division by 0"""
    loss[loss == 0] = 0.00000001
    if np.array_equal(np.full((400,400,3),0.00000001), loss):
      print('loss 0')
    

    """resizing reward to 64,64,3"""    
    reward = -(loss/weight_diff)
    reward = cv2.resize(reward, (64,64))

    self.paintings_per_loop -= 1
    self.state = cv2.resize(self.reconstructed, (64,64))
    self.reward = reward

    if self.paintings_per_loop <= 0:
      done = True
    else:
      done = False
    self.loss = loss
    info = {}
    return self.state, reward, done, info


  def render(self, mode = None):
    """utility function to render environment at any time step"""
    original_img = (cv2.resize(self.state, (400,400)))
    img = PIL.Image.fromarray(((original_img)*255).astype('uint8'))
    reconstructed = ((self.reconstructed*255).astype('uint8')).squeeze()
    if mode == 'human':
      d = PIL.ImageDraw.Draw(img)
      font = ImageFont.load_default()
      y = 0
      _, height = d.textsize("W", font)
      d.text((0, y), f"Reward: {tf.math.reduce_mean(self.reward)}", fill=(0, 255, 0))
      y+=height
      d.text((0, y), f"Loss: {tf.math.reduce_mean(self.loss)}", fill=(0, 255, 0))
      y+=height
      reconstructed = ((self.reconstructed*255).astype('uint8')).squeeze()
      img = np.array(img)
      img = np.concatenate((img, reconstructed), axis = 1)
    else:
      img = np.array(img)
      img = np.concatenate((img, reconstructed), axis = 1)      
    return (np.array(img))


  def reset(self):
    #self.state = np.zeros((100,100,3))
    self.paintings_per_loop = self.paintings_per_loop_original
    return self.state

In [7]:
"""registering the gym environment"""
gym.envs.registration.register(
    id='painting_environment-v1',
    entry_point=f'{__name__}:Painting_Env',
)

In [8]:
class ppo_memory:
  def __init__(self, batch_size):
    self.states = []
    self.probabilities = [] 
    self.terminal_memory = []
    self.values = []
    self.actions = []
    self.rewards = []
    self.batch_size = batch_size

  """storing memories"""
  def store_memory(self, state, action, probabilities, values, reward, done):
    self.states.append(state)
    self.actions.append(action)
    self.probabilities.append(probabilities)
    self.values.append(values)
    self.rewards.append(reward)
    self.terminal_memory.append(done)

  """clearing all memory, happens after a completed learning cycle"""
  def clear_memory(self):
    self.states = []
    self.probabilities=[]
    self.rewards = []
    self.terminal_memory = []
    self.values = []
    self.actions = []

  """returning states, actions, values (for the critic), probabilities, and the indices of these items"""
  def generate_batches(self):
    number_states = len(self.states)
    batch_start_points = np.arange(0, number_states, self.batch_size)

    indices = np.arange(number_states, dtype=np.int64)

    batches = [indices[i:i+self.batch_size] for i in batch_start_points]

    return np.array(self.states), np.array(self.actions), np.array(self.probabilities), np.array(self.values), np.array(self.rewards),\
    np.array(self.terminal_memory), batches



In [9]:
"""custom keras layer for the actor network, returns a gaussian distribution over every
colour channel for every pixel"""
class gaussian_dist(tf.keras.layers.Layer):
  def call(self, inputs) -> Tensor:
    means, stddevs = inputs
    dist = tfp.distributions.Normal(loc = means, scale = stddevs**2)
    return dist

In [10]:
class actor_network(tf.keras.Model):
  
  def __init__(self, n_actions, conv_layer_params=None, conv_transpose_params=None, lstm_size = None, midpoint_fc_params = None, name = 'actor',
               batch_size = 16):
    
    super(actor_network, self).__init__()

    self.n_actions = n_actions
    self.model_name = name

    self.checkpoint_file = self.model_name+'_ppo.h5'
    self.midpoint_fc_params = midpoint_fc_params
    self.conv_layer_params = conv_layer_params
    flatten_dims = None
    """finding the flattened dimensions for the bottleneck of the actor (after the LSTM section)"""
    for i in range(0,len(conv_layer_params)):
      if not flatten_dims:
        flatten_dims = conv_layer_params[i][2]
      else:
        flatten_dims *= conv_layer_params[i][2]

    self.reshape_dims = np.array([n_actions[0],n_actions[1]])/flatten_dims
    self.reshape_dims = self.reshape_dims.astype(int)
    flatten_dims = (self.reshape_dims[0]*self.reshape_dims[1])*conv_layer_params[-1][0]
    self.flatten_dims = flatten_dims

    input_encoder = []

    """building an input encoder, which is convolutional"""
    for config in conv_layer_params:
      (filters, kernel_size, strides) = config
      input_encoder.append(
            tf.keras.layers.Conv2D(
                filters=filters,
                kernel_size=kernel_size,
                strides=strides,
                dilation_rate=(1,1),
                padding = 'same'))
      input_encoder.append(tf.keras.layers.BatchNormalization(momentum=0.99))
      input_encoder.append(tf.keras.layers.LeakyReLU())
    
    """building a two layer LSTM network"""
    lstm_network = []
    for i in range(2):
      lstm_network.append(tf.keras.layers.LSTM(128, return_sequences = True, stateful = True))

    lstm_network.append(tf.keras.layers.Dense(flatten_dims))
    lstm_network.append(tf.keras.layers.LeakyReLU())
    
    output_encoder = []
    
    """building an output decoder, which is made of deconvolutional layers"""
    for config in conv_transpose_params:
      (filters, kernel_size, strides) = config
      output_encoder.append(
            tf.keras.layers.Conv2DTranspose(
                filters=filters,
                kernel_size=kernel_size,
                strides=strides,
                dilation_rate=(1,1),
                padding = 'same'))
      output_encoder.append(tf.keras.layers.BatchNormalization(momentum=0.99))
      output_encoder.append(tf.keras.layers.LeakyReLU())
    
    """creating the distributions necessary for the custom layer"""
    output_encoder_means = tf.keras.layers.Conv2DTranspose(filters = 3, kernel_size = (1,1), strides = (1,1), dilation_rate=(1,1), padding='same', activation = 'sigmoid')
    output_encoder_stddevs = tf.keras.layers.Conv2DTranspose(filters = 3, kernel_size = (1,1), strides = (1,1), dilation_rate=(1,1), padding='same',activation='sigmoid')
    output_encoder_dist = gaussian_dist()

    self.input_encoder = input_encoder
    self.lstm_network = lstm_network
    self.output_encoder = output_encoder
    self.output_encoder_means = output_encoder_means
    self.output_encoder_stddevs = output_encoder_stddevs
    self.output_encoder_dist = output_encoder_dist

  def call(self, state):

      """reshaping data to make it appropriate for an LSTM network"""
      if tf.rank(state) == 3:
        state = tf.reshape(state, (1,1,self.n_actions[0],self.n_actions[1],self.n_actions[2]))
      else:
        state = tf.reshape(state, (1,len(state),self.n_actions[0],self.n_actions[1],self.n_actions[2]))
      
      action_value = self.input_encoder[0](state)
      input_shape = np.shape(action_value)
      """putting data through the input encoder"""
      for i in range(1,len(self.input_encoder)):
        action_value = self.input_encoder[i](action_value)
      """reshaping data to be suitable for lstm network"""
      action_value = tf.reshape(action_value, (1,input_shape[1],self.flatten_dims))
      for layer in self.lstm_network:
        action_value = layer(action_value)
      """reshaping data to be suitable for the deconvolutional layers"""
      action_value = tf.reshape(action_value, (input_shape[1],self.reshape_dims[0],self.reshape_dims[1],256))
      for layer in self.output_encoder:
        action_value = layer(action_value)
      """generating and returning distribution"""
      action_value_means = self.output_encoder_means(action_value)
      action_value_stddevs = self.output_encoder_stddevs(action_value)
      dist_channel = self.output_encoder_dist([action_value_means, action_value_stddevs])


      return dist_channel

In [11]:
class critic_network(tf.keras.Model):

  def __init__(self, n_actions, conv_layer_params=None, conv_transpose_params=None, lstm_size = None, midpoint_fc_params = None, name = 'critic'):
    
    super(critic_network, self).__init__()

    """n_actions is the dimensions of the actions available, so in this case 64,64,3"""
    self.n_actions = n_actions
    self.model_name = name

    self.checkpoint_file = self.model_name+'_ppo.h5'
    self.midpoint_fc_params = midpoint_fc_params
    self.conv_layer_params = conv_layer_params
    flatten_dims = None
    """finding the flattened dimensions for the bottleneck of the critic (after the LSTM section)"""
    for i in range(0,len(conv_layer_params)):
      if not flatten_dims:
        flatten_dims = conv_layer_params[i][2]
      else:
        flatten_dims *= conv_layer_params[i][2]

    self.reshape_dims = np.array([n_actions[0],n_actions[1]])/flatten_dims
    self.reshape_dims = self.reshape_dims.astype(int)
    flatten_dims = (self.reshape_dims[0]*self.reshape_dims[1])*conv_layer_params[-1][0]
    self.flatten_dims = flatten_dims

    """building an input encoder, which is convolutional"""
    input_encoder = []
    for config in conv_layer_params:
      (filters, kernel_size, strides) = config
      input_encoder.append(
            tf.keras.layers.Conv2D(
                filters=filters,
                kernel_size=kernel_size,
                strides=strides,
                dilation_rate=(1,1),
                activation=tf.keras.activations.relu,
                padding = 'same'))
      input_encoder.append(tf.keras.layers.BatchNormalization(momentum=0.99))
      input_encoder.append(tf.keras.layers.LeakyReLU())


    lstm_network = []
    """building a two layer LSTM network"""
    for i in range(2):
      lstm_network.append(tf.keras.layers.LSTM(128, return_sequences = True, stateful = True))

    lstm_network.append(tf.keras.layers.Dense(flatten_dims))
    lstm_network.append(tf.keras.layers.LeakyReLU())

    output_encoder = []
    """building an output decoder, which is made of deconvolutional layers"""
    for config in conv_transpose_params:
      (filters, kernel_size, strides) = config
      output_encoder.append(
            tf.keras.layers.Conv2DTranspose(
                filters=filters,
                kernel_size=kernel_size,
                strides=strides,
                dilation_rate=(1,1),
                padding = 'same'))
      output_encoder.append(tf.keras.layers.BatchNormalization(momentum=0.99))
      output_encoder.append(tf.keras.layers.LeakyReLU())
      #output_encoder.append(tf.keras.layers.Dropout(0.2))
    
    output_encoder.append(tf.keras.layers.Conv2DTranspose(filters =3, kernel_size = (1,1), strides = (1,1), dilation_rate=(1,1),padding = 'same', activation='sigmoid'))
    
    self.input_encoder = input_encoder
    self.lstm_network = lstm_network
    self.output_encoder = output_encoder
    

  def call(self, state):
      """reshaping data to make it appropriate for an LSTM network"""
      if tf.rank(state) == 3:
        state = tf.reshape(state, (1,1,self.n_actions[0],self.n_actions[1],self.n_actions[2]))
      else:
        state = tf.reshape(state, (1,len(state),self.n_actions[0],self.n_actions[1],self.n_actions[2]))
      action_value = self.input_encoder[0](state)
      input_shape = np.shape(action_value)
      """putting data through the input encoder"""
      for i in range(1,len(self.input_encoder)):
        action_value = self.input_encoder[i](action_value)
      """reshaping data to be suitable for lstm network"""
      action_value = tf.reshape(action_value, (1,input_shape[1],self.flatten_dims))
      for layer in self.lstm_network:
        action_value = layer(action_value)
      """reshaping data to be suitable for the deconvolutional layers"""
      action_value = tf.reshape(action_value, (input_shape[1],self.reshape_dims[0],self.reshape_dims[1],256))
      for layer in self.output_encoder:
        action_value = layer(action_value)

      return action_value

In [12]:
class ppo_agent:
  def __init__(self, n_actions, conv_layer_params, conv_transpose_params, gamma = 0.5, alpha = 0.0003, policy_clip = 0.2, batch_size = 8,
               n_epochs = 5, gae_lambda = 0.95, N = 10):
    
    self.gamma = gamma
    self.n_actions = n_actions
    self.alpha = alpha
    self.policy_clip = policy_clip
    self.batch_size = batch_size
    self.N = N
    self.n_epochs = n_epochs
    self.gae_lambda = gae_lambda

    self.actor = actor_network(n_actions=(64,64,3), conv_layer_params=conv_layer_params, conv_transpose_params=conv_transpose_params, name=os.path.join('experimental_condition_four','actor_network'))
    self.critic = critic_network(n_actions=(64,64,3), conv_layer_params=conv_layer_params, conv_transpose_params=conv_transpose_params, name=os.path.join('experimental_condition_four','critic_network'))
    
    self.actor.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=alpha))

    self.critic.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=alpha))

    self.memory = ppo_memory(self.batch_size)

  def remember(self, state, action, probs, vals, reward, done):
    self.memory.store_memory(state, action, probs, vals, reward, done)

  def save_models(self):
    print('saving models')
    self.actor.save_weights(self.actor.checkpoint_file)
    self.critic.save_weights(self.critic.checkpoint_file)
    vae.autoencoder.save_weights(os.path.join('experimental_condition_four','updated_appraiser.h5'))
    


  def load_models(self):
    print('loading models')
    self.actor.load_weights(self.actor.checkpoint_file)
    self.critic.load_weights(self.critic.checkpoint_file)
    vae.autoencoder.load_weights(os.path.join('experimental_condition_four','updated_appraiser.h5'))


  def choose_action(self, observation):
    actions = self.actor(observation)
    img = (actions.sample())
    action = img

    value = self.critic(observation)

    probs = actions.log_prob(img).numpy()

    return (action.numpy()).astype(np.float32),\
    probs,\
    value.numpy()

  
  def learn(self):
    for _ in range(n_epochs):
      
      state_array, action_array, old_probs_array,\
       vals_array, reward_array, done_array, batches =  self.memory.generate_batches()
      
      
      values = tf.squeeze(vals_array).numpy()


      advantage = np.zeros(shape=np.shape(reward_array), dtype=np.float32)

      """calculating advantage for future time steps from each point in batch"""
      for x in range(len(reward_array)-1):
        discount = 1
        advantage_t = 0
        for y in range(x, len(reward_array)-1):
           advantage_t += discount*(reward_array[y] + self.gamma*values[y+1]*\
                            (1-int(done_array[y])) - values[y])
           discount *= self.gamma*self.gae_lambda
        advantage[x] = advantage_t

      """training per batch"""
      for batch in batches:
        with tf.GradientTape() as t1, tf.GradientTape() as t2:
          batch = np.array(batch)
          states = tf.convert_to_tensor(state_array[batch])
          old_probs = tf.convert_to_tensor(old_probs_array[batch])

          actions = tf.convert_to_tensor(action_array[batch])
          """finding the distributions and values per batch of states"""
          distribution = self.actor(states)

          critic_value = tf.squeeze(self.critic(states))
          
          """finding new probability and comparing it to old"""
          new_probs = distribution.log_prob(actions)

          probability_ratio = tf.exp(new_probs) / tf.exp(old_probs)

          weighted_probs = advantage[batch] * probability_ratio
          """clipping values"""
          weighted_clipped_probs = tf.clip_by_value(weighted_probs, 1-self.policy_clip, 1+self.policy_clip)*advantage[batch]

          returns = advantage[batch]+values[batch]
        
          """calculating losses"""
          critic_loss = ((returns - critic_value)**2)
          critic_loss = tf.math.reduce_mean(critic_loss)

          actor_loss = tf.math.reduce_mean(-tf.math.minimum(weighted_probs, weighted_clipped_probs))
          total_loss = actor_loss + 0.5*critic_loss
          

          """training actor"""
          actor_gradient = t2.gradient(total_loss, self.actor.trainable_variables)
          self.actor.optimizer.apply_gradients(zip(actor_gradient, self.actor.trainable_variables))


          """training critic"""
          critic_gradient = t1.gradient(total_loss, self.critic.trainable_variables)
          self.critic.optimizer.apply_gradients(zip(critic_gradient, self.critic.trainable_variables))


    self.memory.clear_memory()



Below cell is the equivalent of a config file

In [13]:
"""change this to reload models later"""
load_models = True
"""frequency with which to display outputs"""
disp_frequency = 25
"""whether to display progress in terms of score"""
verbose = False

In [14]:
"""creating environment"""
env = gym.make('painting_environment-v1')
processed_imgs_two = np.zeros((1,400,800,3), dtype = int)
"""number of actions taken between learning"""
N = 10
"""number of epochs per learning iteration"""
n_epochs = 8
"""learning rate for actor and critic"""
alpha = 0.0001
"""batch size for actor/critic training"""
batch_size = 8
learn_iters = 0
avg_score = 0
n_steps = 0
a = 0
agent = ppo_agent(n_actions=env.action_space.shape, 
                      conv_layer_params =[(64, 2, 2), (128, 2, 2),(128,1,1),(128,1,1),(256,1,1)], 
                      conv_transpose_params=[(256,1,1),(128,1,1),(128,1,1),(128,2,2),(64,2,2)],
                      batch_size = batch_size, n_epochs = n_epochs, alpha = alpha)
if load_models:
        agent.actor(np.zeros((1,64,64,3)))
        agent.critic(np.zeros((1,64,64,3)))
        agent.load_models()
"""how many episodes"""
n_episodes = 1000
best_score = env.reward_range[0]
steps_since_improvement = 0
score_history = []
avg_score_history = []#list(np.load(os.path.join('condition_three','score_history_weights_invis_reviewed.npy')))

loading models


In [15]:
for i in range(n_episodes):
  observation = env.reset()
  done = False
  score = 0
  while not done:
    action, probs, value = agent.choose_action(observation.reshape(1,64,64,3))
    observation_, reward, done, info = env.step(action)
    n_steps += 1
    score = (tf.reduce_mean(reward)).numpy()
    score_history.append(score)
    agent.remember(observation,\
                   action, probs, value, reward, done)
    
    
    if n_steps % N == 0:
      print('learning')
      agent.learn()
      learn_iters += 1
    
    if n_steps % disp_frequency == 0:
      print('rendering')
      plt.figure()
      plt.imshow(env.render())
      plt.pause(1)
      plt.figure()
      plt.imshow(action.squeeze())
      plt.pause(1)
      print(score)
        
    observation = observation_
  
  agent.save_models()
  avg_score = np.mean(score_history)
  avg_score_history.append(avg_score)
  score_history = []
  if verbose:
      print('episode ',str(i), 'score %1f' %score, 'avg_score %1f' %avg_score, 'time steps %1f' %n_steps, '\n learning iters %1f' %learn_iters)

learning
learning


KeyboardInterrupt: 